In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data35095  dataset  train  val


In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
# !mkdir /home/aistudio/external-libraries
# !pip install beautifulsoup4 -t /home/aistudio/external-libraries
!pip install paddlepaddle-gpu
import paddle as paddle
# Initialize PaddlePaddle.
with_gpu = os.getenv('WITH_GPU', '1') != '0'

Looking in indexes: https://mirror.baidu.com/pypi/simple/


In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [ ]:
import paddle
print("paddle " + paddle.__version__)

paddle 2.0.1


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [ ]:
device = paddle.set_device("gpu")  # 指定设备
print("using {} device.".format(device))

using CUDAPlace(0) device.


In [ ]:
def calculate_fee(distance_travelled):
    return 10 + 2 * distance_travelled

for x in [1.0, 3.0, 5.0, 9.0, 10.0, 20.0]:
    print(calculate_fee(x))
    
import paddle
print("paddle " + paddle.__version__)
x_data = paddle.to_tensor([[1.], [3.0], [5.0], [9.0], [10.0], [20.0]])
y_data = paddle.to_tensor([[12.], [16.0], [20.0], [28.0], [30.0], [50.0]])
linear = paddle.nn.Linear(in_features=1, out_features=1)
w_before_opt = linear.weight.numpy().item()
b_before_opt = linear.bias.numpy().item()

print("w before optimize: {}".format(w_before_opt))
print("b before optimize: {}".format(b_before_opt))
mse_loss = paddle.nn.MSELoss()
sgd_optimizer = paddle.optimizer.SGD(learning_rate=0.001, parameters = linear.parameters())
total_epoch = 5000
for i in range(total_epoch):
    y_predict = linear(x_data)
    loss = mse_loss(y_predict, y_data)
    loss.backward()
    sgd_optimizer.step()
    sgd_optimizer.clear_grad()

    if i%1000 == 0:
        print("epoch {} loss {}".format(i, loss.numpy()))

print("finished training， loss {}".format(loss.numpy()))
w_after_opt = linear.weight.numpy().item()
b_after_opt = linear.bias.numpy().item()

print("w after optimize: {}".format(w_after_opt))
print("b after optimize: {}".format(b_after_opt))
print("hello paddle")

12.0
16.0
20.0
28.0
30.0
50.0
paddle 2.0.1
w before optimize: 0.9323206543922424
b before optimize: 0.0
epoch 0 loss [387.86243]
epoch 1000 loss [8.180787]
epoch 2000 loss [1.8291637]
epoch 3000 loss [0.40899396]
epoch 4000 loss [0.09144972]
finished training， loss [0.02047826]
w after optimize: 2.0182228088378906
b after optimize: 9.766996383666992
hello paddle


In [ ]:
import zipfile
with zipfile.ZipFile("data/data35095/train.zip") as zf:
   zf.extractall()

In [ ]:
! mv train data/dataset

In [ ]:
import os
from shutil import copy, rmtree
import random


def mk_file(file_path: str):
    if os.path.exists(file_path):
        # 如果文件夹存在，则先删除原文件夹在重新创建
        rmtree(file_path)
    os.makedirs(file_path)



# 保证随机可复现
random.seed(0)

# 将数据集中10%的数据划分到验证集中
split_rate = 0.1

# 指向你解压后的flower_photos文件夹
# cwd = os.getcwd()
data_root = "data/"
origin_garbage_path = "data/dataset"
assert os.path.exists(origin_garbage_path)
garbage_class = [cla for cla in os.listdir(origin_garbage_path)
                if os.path.isdir(os.path.join(origin_garbage_path, cla))]

# 建立保存训练集的文件夹
train_root = os.path.join(data_root, "train")
mk_file(train_root)
for cla in garbage_class:
    # 建立每个类别对应的文件夹
    mk_file(os.path.join(train_root, cla))

# 建立保存验证集的文件夹
val_root = os.path.join(data_root, "val")
mk_file(val_root)
for cla in garbage_class:
    # 建立每个类别对应的文件夹
    mk_file(os.path.join(val_root, cla))

for cla in garbage_class:
    cla_path = os.path.join(origin_garbage_path, cla)
    images = os.listdir(cla_path)
    num = len(images)
    # 随机采样验证集的索引
    eval_index = random.sample(images, k=int(num*split_rate))
    for index, image in enumerate(images):
        if image in eval_index:
            # 将分配至验证集中的文件复制到相应目录
            image_path = os.path.join(cla_path, image)
            new_path = os.path.join(val_root, cla)
            copy(image_path, new_path)
        else:
            # 将分配至训练集中的文件复制到相应目录
            image_path = os.path.join(cla_path, image)
            new_path = os.path.join(train_root, cla)
            copy(image_path, new_path)
        print("\r[{}] processing [{}/{}]".format(cla, index+1, num), end="")  # processing bar
    print()

print("processing done!")

[36] processing [255/255]
[14] processing [347/347]
[34] processing [385/385]
[11] processing [726/726]
[30] processing [311/311]
[35] processing [341/341]
[10] processing [377/377]
[26] processing [341/341]
[2] processing [269/269]
[24] processing [308/308]
[21] processing [647/647]
[33] processing [312/312]
[4] processing [377/377]
[25] processing [540/540]
[37] processing [312/312]
[5] processing [279/279]
[0] processing [232/232]
[16] processing [342/342]
[28] processing [372/372]
[20] processing [216/216]
[13] processing [399/399]
[19] processing [302/302]
[18] processing [352/352]
[32] processing [270/270]
[29] processing [406/406]
[38] processing [381/381]
[23] processing [299/299]
[6] processing [385/385]
[3] processing [75/75]
[22] processing [365/365]
[27] processing [526/526]
[12] processing [321/321]
[39] processing [427/427]
[15] processing [409/409]
[31] processing [436/436]
[8] processing [370/370]
[9] processing [379/379]
[17] processing [299/299]
[7] processing [352/35

In [ ]:
import paddle.nn as nn  # 包含了神经网络的主要元素
import paddle

In [ ]:
class AlexNet(nn.Layer):  # 集成module可以直接调用父类函数，module是所有神经网络的基类
    def __init__(self, num_classes=1000):  # 初始化模型
        super(AlexNet, self).__init__()  # 使用父类的方法对子类进行初始化
        self.features = nn.Sequential(  # 一系列的结构打包
            nn.Conv2D(3, 48, kernel_size=11, stride=4, padding=2, weight_attr=nn.initializer.KaimingNormal()),  # input[3, 224, 224]  output[48, 55, 55]
            # (224-11+4)/4+1=55
            nn.ReLU(),
            nn.MaxPool2D(kernel_size=3, stride=2),                  # output[48, 27, 27]
            # (55-3)/2+1=27
            nn.Conv2D(48, 128, kernel_size=5, padding=2, weight_attr=nn.initializer.KaimingNormal()),           # output[128, 27, 27]
            # (27+4-5)+1=27
            nn.ReLU(),
            nn.MaxPool2D(kernel_size=3, stride=2),                  # output[128, 13, 13]
            # (27-3)/2+1=13
            nn.Conv2D(128, 192, kernel_size=3, padding=1, weight_attr=nn.initializer.KaimingNormal()),          # output[192, 13, 13]
            # (13+2-3)+1=13
            nn.ReLU(),
            nn.Conv2D(192, 192, kernel_size=3, padding=1, weight_attr=nn.initializer.KaimingNormal()),          # output[192, 13, 13]
            # (13+2-3)+1=13
            nn.ReLU(),
            nn.Conv2D(192, 128, kernel_size=3, padding=1, weight_attr=nn.initializer.KaimingNormal()),          # output[128, 13, 13]
            # (13+2-3)+1=13
            nn.ReLU(),
            nn.MaxPool2D(kernel_size=3, stride=2),                  # output[128, 6, 6]
            # (13-3)/2+1=6
        )
        self.classifier = nn.Sequential(  # 全连接层分类器
            nn.Dropout(p=0.5),  # 随即失活的比例
            nn.Linear(128 * 6 * 6, 2048, weight_attr=nn.initializer.KaimingNormal()),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(2048, 2048, weight_attr=nn.initializer.KaimingNormal()),
            nn.ReLU(),
            nn.Linear(2048, num_classes, weight_attr=nn.initializer.KaimingNormal()),
        )
    def forward(self, x):
        x = self.features(x)
        x = paddle.flatten(x, start_axis=1)  # 展平
        x = self.classifier(x)
        return x

In [ ]:
import os
import json
import time

import paddle
import paddle.nn as nn

from paddle.vision import transforms
from paddle.vision import datasets
import paddle.optimizer as optim
from paddle.static import InputSpec

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
2021-03-29 09:46:27,634 - INFO - font search path ['/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf', '/opt/co

In [ ]:
data_transform = {
    # 训练集预处理
    "train": transforms.Compose([transforms.RandomResizedCrop(224),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
    # 验证集预处理
    "val": transforms.Compose([transforms.Resize((224, 224)),  # cannot 224, must (224, 224)
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])}

In [ ]:
 data_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))  # get data root path
# image_path = os.path.join(data_root)  # flower data set path
# assert os.path.exists(image_path), "{} path does not exist.".format(image_path)
train_dataset = datasets.DatasetFolder(root="data/train",
                                     transform=data_transform["train"])
train_num = len(train_dataset)
garbage_list = train_dataset.class_to_idx
cla_dict = dict((val, key) for key, val in garbage_list.items())  # key和val反一下
# write dict into json file



In [ ]:
json_str = json.dumps(cla_dict, indent=4)
with open('class_indices.json', 'w') as json_file:
    json_file.write(json_str)

In [19]:

batch_size = 128
train_loader = paddle.io.DataLoader(train_dataset,
                                    batch_size=batch_size, shuffle=True,
                                    num_workers=0)
validate_dataset = datasets.DatasetFolder(root="data/val",
                                          transform=data_transform["val"])
val_num = len(validate_dataset)
validate_loader = paddle.io.DataLoader(validate_dataset,
                                        batch_size=batch_size, shuffle=True,
                                        num_workers=0)
print("using {} images for training, {} images for validation.".format(train_num,
                                                                       val_num))

using 12980 images for training, 1422 images for validation.


In [20]:
net = AlexNet(num_classes=40)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(parameters=net.parameters(), learning_rate=0.0001)

epochs = 100
save_path = 'AlexNet.pth'
best_acc = 0.0
train_steps = len(train_loader)
for epoch in range(epochs):
    # train
    net.train()  # 启用dropout
    running_loss = 0.0
    t1 = time.perf_counter()
    for step, data in enumerate(train_loader, start=0):
        images, labels = data
        outputs = net(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.clear_grad() # 梯度置零

        # print statistics
        running_loss += loss.numpy()[0]
        rate = (step + 1) / len(train_loader)
        a = "*" * int(rate * 50)
        b = "." * int((1 - rate) * 50)
        loss_num = loss.numpy()[0]
        print("\rtrain loss: {:^3.0f}%[{}->{}]{:.3f}".format(int(rate * 100), a, b, loss_num), end=" ")
    print()
    print(time.perf_counter()-t1)

    # validate
    net.eval()  # 关闭dropout
    acc = 0.0  # accumulate accurate number / epoch
    with paddle.no_grad():  # 禁止跟踪
        for val_data in validate_loader:
            val_images, val_labels = val_data
            outputs = net(val_images)
            predict_y = outputs.numpy().argmax(axis=1)
            acc += (val_labels.astype("int").numpy()==outputs.numpy().argmax(axis=1).astype("int")).sum()

        val_accurate = acc / val_num
        print('[epoch %d] train_loss: %.3f  val_accuracy: %.3f' %
              (epoch + 1, running_loss / train_steps, val_accurate))

        if val_accurate > best_acc:
            best_acc = val_accurate
            paddle.save(net.state_dict(), save_path)

print('Finished Training')

train loss: 100%[**************************************************->]3.357 
89.20627645403147
[epoch 1] train_loss: 3.627  val_accuracy: 0.122
train loss: 100%[**************************************************->]3.296 
88.93507681041956
[epoch 2] train_loss: 3.350  val_accuracy: 0.130
train loss: 100%[**************************************************->]3.209 
88.60019026696682
[epoch 3] train_loss: 3.231  val_accuracy: 0.186
train loss: 100%[**************************************************->]3.084 
89.09733299165964
[epoch 4] train_loss: 3.151  val_accuracy: 0.190
train loss: 100%[**************************************************->]2.972 
88.08202955126762
[epoch 5] train_loss: 3.064  val_accuracy: 0.202
train loss: 100%[**************************************************->]2.863 
88.71006862819195
[epoch 6] train_loss: 2.997  val_accuracy: 0.221
train loss: 100%[**************************************************->]2.903 
89.40387020260096
[epoch 7] train_loss: 2.940  val_accuracy

In [21]:
with zipfile.ZipFile("data/data35095/test.zip") as zf:
   zf.extractall()

In [22]:
paddle.jit.save(
    layer=net,
    path="model",
    input_spec=[InputSpec(shape=[1, 3, 224, 224], dtype='float32')])

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and


In [ ]:
data_transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# load image
with open("test.txt","w") as f:
    for i in range(1,401):
        img_path = "test/test"+str(i)+".jpg"
        assert os.path.exists(img_path), "file: '{}' dose not exist.".format(img_path)
        img = Image.open(img_path)

        paddle.disable_static()
        # plt.imshow(img)
        # [N, C, H, W]
        img = data_transform(img)
        # expand batch dimension
        img = paddle.unsqueeze(img, axis=0)

        # read class_indict
        json_path = 'class_indices.json'
        assert os.path.exists(json_path), "file: '{}' dose not exist.".format(json_path)

        json_file = open(json_path, "r")
        class_indict = json.load(json_file)

        # create model
        model = AlexNet(num_classes=40)

        # load model weights
        # weights_path = "AlexNet.pth"
        # assert os.path.exists(weights_path), "file: '{}' dose not exist.".format(weights_path)
        model = paddle.jit.load("model")
        # model.load_state_dict(torch.load(weights_path))

        model.eval()
        with paddle.no_grad():
            # predict class
            output = paddle.squeeze(model(img))
            predict = nn.Softmax(axis=0)(output)
            predict_cla = paddle.argmax(predict).numpy()[0]

        # print_res = "class: {}   prob: {:.3}".format(class_indict[str(predict_cla)],
                                                    #predict.numpy()[predict_cla])
        # print(print_res)
    
        f.write(class_indict[str(predict_cla)]+"\n")

        rateTest = i / 400
        aTest = "*" * int(rateTest * 50)
        bTest = "." * int((1 - rateTest) * 50)
        print("\rprocess: {:d}% [{}->{}]".format(int(rateTest * 100), aTest, bTest), end=" ")



process: 17% [********->.........................................]  